# Running an MSTIS simulation

Now we will use the initial trajectories we obtained from bootstrapping to run an MSTIS simulation. This will show both how objects can be regenerated from storage and how regenerated equivalent objects can be used in place of objects that weren't stored.

Tasks covered in this notebook:
* Loading OPS objects from storage
* Ways of assigning initial trajectories to initial samples
* Setting up a path sampling simulation with various move schemes
* Visualizing trajectories while the path sampling is running

In [1]:
%matplotlib inline
import openpathsampling as paths
import numpy as np
import math

In [2]:
# Imports for plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.legend_handler import HandlerLine2D

### Loading things from storage

First we'll reload some of the stuff we stored before. Of course, this starts with opening the file.

In [3]:
old_store = paths.AnalysisStorage("ala.nc")

A lot of information can be recovered from the old storage, and so we don't have the recreate it. However, we did not save our network, so we'll have to create a new one. Since the network creates the ensembles, that means we will have to translate the trajectories from the old ensembles to new ensembles.

In [4]:
print "PathMovers:", len(old_store.pathmovers)
print "Engines:", len(old_store.engines)
print "Samples:", len(old_store.samples)
print "Ensembles:", len(old_store.ensembles)
print "SampleSets:", len(old_store.samplesets)
print "Snapshots:", len(old_store.snapshots)
print "Networks:", len(old_store.networks)

PathMovers: 0
Engines: 2
Samples: 22
Ensembles: 291
SampleSets: 1
Snapshots: 5742
Networks: 1


Loading from storage is very easy. Each store is a list. We take the 0th snapshot as a template (it doesn't actually matter which one) for the next storage we'll create. There's only one engine stored, so we take the only one.

In [5]:
template = old_store.snapshots[0]

In [6]:
engine = old_store.engines.find_first('default')

In [7]:
mstis = old_store.networks[0]

Now we need to set up real trajectories that we can use for each of these. We can start by loading the stored sample set.

In [8]:
# load the sample set we have saved before
old_sampleset = old_store.samplesets[0]

#### About `Sample`s

The OPS object called `Sample` is used to associate a trajectory with a replica ID and an ensemble. The trajectory needs to be associated with an ensemble so we know how to get correct statistics from the many ensembles that we might be sampling simultaneously. The trajectory needs to be associated with a replica ID so that replica exchange approaches can be analyzed.



Since the ensembles in our MSTIS network are not the exact ensemble objects that we saved our samples with (they were rebuilt), we still need a way to identify which of the new ensembles to associate them with.

There are two main ways to do this. The first is to take one trajectory, and associate it with as many ensembles as possible. If your first path comes from a TPS simulation, that is the approach you'll want to take.

The second approach is better suited to our conditions here: we already have a good trajectory for each ensemble. So we just want to remap our old ensembles to new ones.

### Remapping old ensembles to new ensembles

If your old and new ensembles have the same string representations, then OPS has a function to help you automatically map them. As long as you create the ensembles in the same way, they'll have the same string representation. Note that if you *don't* have the same string representation, you would have to assign trajectories to ensembles by hand (which isn't that hard, but is a bit tedious).

In [9]:
sset = paths.SampleSet(old_sampleset)

In [10]:
sset.sanity_check()
assert(len(sset)==22)

### Setting up special ensembles

Whichever way we initially set up the `SampleSet`, at this point it only contains samples for the main sampling trajectories of each transition. Now we need to put trajectories into various auxiliary ensembles.

#### Multiple state outer ensemble

The multiple state outer ensemble is, in fact, sampled during the bootstrapping. However, it is actually sampled once for every state that shares it. It is very easy to find a trajectory that satisfies the ensemble and to load add that sample to our sample set.

In [11]:
for outer_ens in mstis.special_ensembles['ms_outer']:
    # doesn't matter which we take, so we take the first
    traj = next(s.trajectory for s in old_sampleset if outer_ens(s.trajectory)==True)
    samp = paths.Sample(
            replica=None,
            ensemble=outer_ens,
            trajectory=traj
    )
    # now we apply it and correct for the replica ID
    sset.append_as_new_replica(samp)

In [12]:
sset.sanity_check()
assert(len(sset)==23)

#### Minus interface ensemble

The minus interface ensembles do not yet have a trajectory. We will generate them by starting with same-state trajectories (A-to-A, B-to-B, C-to-C) in each interface, and extending into the minus ensemble.

* check whether the traj is A-to-A
* extend

First we need to make sure that the trajectory in the innermost ensemble of each state also ends in that state. This is necessary so that when we extend the trajectory, it can extends into the minus ensemble.

If the trajectory isn't right, we run a shooting move on it until it is.

In [13]:
for transition in mstis.sampling_transitions:
    innermost_ensemble = transition.ensembles[0]
    shooter = None
    if not transition.stateA(sset[innermost_ensemble].trajectory[-1]):
        shooter = paths.OneWayShootingMover(ensemble=innermost_ensemble,
                                            selector=paths.UniformSelector())
        pseudoscheme = paths.LockedMoveScheme(root_mover=shooter)
        pseudosim = paths.PathSampling(storage=None, 
                                       move_scheme=pseudoscheme, 
                                       globalstate=sset,
                                       engine=engine
                                      )
    while not transition.stateA(sset[innermost_ensemble].trajectory[-1]):
        pseudosim.run(1)
        sset = pseudosim.globalstate

    

Now that all the innermost ensembles are safe to use for extending into a minus interface, we extend them into a minus interface:

In [16]:
minus_samples = []
for transition in mstis.sampling_transitions:
    print transition.name, 
    choices = list()
    while len(choices) == 0:        
        attempt = transition.minus_ensemble.populate_minus_ensemble(
            partial_traj=sset[transition.ensembles[0]].trajectory,
            minus_replica_id=-len(minus_samples)-1,
            engine=engine
        )
        choices = transition.minus_ensemble.split(attempt)
        print len(attempt),

    print len(choices), len(choices[0])
        
    minus_samples.append(
        paths.Sample(
            replica=attempt.replica,
            trajectory=choices[0],
            ensemble=transition.minus_ensemble
        )        
    )
    
sset = sset.apply_samples(minus_samples)

 Out C 41 1 21
Out E 151 1 143
Out A 63 1 48
Out B 30 1 30
Out F 24 1 24
Out D 20 1 16


In [17]:
sset.sanity_check()
assert(len(sset)==29)

In [18]:
for s in sset:
    print s.replica, len(s.trajectory), s.ensemble.name, s.ensemble(s.trajectory)

0 10 I'face 0 True
1 27 I'face 1 True
2 25 I'face 2 True
3 25 [TISEnsemble] True
4 18 I'face 0 True
5 33 I'face 1 True
6 33 [TISEnsemble] True
7 17 I'face 0 True
8 17 I'face 1 True
9 23 I'face 2 True
10 23 [TISEnsemble] True
11 120 I'face 0 True
12 32 I'face 1 True
13 34 I'face 2 True
14 34 [TISEnsemble] True
15 10 I'face 0 True
16 20 I'face 1 True
17 20 [TISEnsemble] True
18 16 I'face 0 True
19 32 I'face 1 True
20 20 I'face 2 True
21 20 [TISEnsemble] True
22 27 [UnionEnsemble] True
-1 21 [MinusInterfaceEnsemble] True
-2 143 [MinusInterfaceEnsemble] True
-3 48 [MinusInterfaceEnsemble] True
-4 30 [MinusInterfaceEnsemble] True
-5 24 [MinusInterfaceEnsemble] True
-6 16 [MinusInterfaceEnsemble] True


In [19]:
samp = sset[-3]
print samp.trajectory.summarize_by_volumes_str(
            {"A" : samp.ensemble.state_vol,
             "I" : ~samp.ensemble.state_vol & samp.ensemble.innermost_vol,
             "X" : ~samp.ensemble.innermost_vol})
print samp.ensemble.split(samp.trajectory)

A-X-I-A-X-I-X-I-X-I-X-I-X-I-X-I-A
[Trajectory[48]]


## Equilibration

In molecular dynamics, you need to equilibrate if you don't start with an equilibrium frame (e.g., if you start with solvent molecules on a grid, your system should equilibrate before you start taking statistics). Similarly, if you start with a set of paths which are far from the path ensemble equilibrium, you need to equilibrate. This could either be because your trajectories are not from the real dynamics (generated with metadynamics, high temperature, etc.) or because your trajectories are not representative of the path ensemble (e.g., if you put transition trajectories into all interfaces).

As with MD, running equilibration can be the same process as running the total simulation. However, in path sampling, it doesn't have to be: we can equilibrate without replica exchange moves or path reversal moves, for example. In the example below, we create a `MoveScheme` that only includes shooting movers.

In [20]:
equil_scheme = paths.MoveScheme(mstis)
import openpathsampling.analysis.move_strategy as strat
equil_scheme.append([strat.OneWayShootingStrategy(), strat.OrganizeByMoveGroupStrategy()])

In [21]:
equilibration = paths.PathSampling(
    storage=None,
    globalstate=sset,
    engine=engine,
    move_scheme=equil_scheme
)

In [22]:
equilibration.run(5)

Working on Monte Carlo cycle number 5.

DONE! Completed 5 Monte Carlo cycles.



In [23]:
sset = equilibration.globalstate

## Running RETIS

Now we run the full calculation. Up to here, we haven't been storing any of our results. This time, we'll start a storage object, and we'll save the network we've created. Then we'll run a new `PathSampling` calculation object.

In [24]:
# logging creates ops_output.log file with details of what the calculation is doing
#import logging.config
#logging.config.fileConfig("logging.conf", disable_existing_loggers=False)

In [25]:
storage = paths.storage.Storage("weina_production.nc", "w", template)

In [26]:
mstis_calc = paths.PathSampling(
    storage=storage,
    globalstate=sset,
    engine=engine,
    move_scheme=paths.DefaultScheme(mstis)
)
mstis_calc.save_frequency = 10

The next block sets up a live visualization. This is optional, and only recommended if you're using OPS interactively (which would only be for very small systems). Some of the same tools can be used to play back the behavior after the fact if you want to see the behavior for more complicated systems. You can create a background (here we use the PES contours), and the visualization will plot the trajectories.

In [28]:
phi = old_store.cvs.find_first('phi')
psi = old_store.cvs.find_first('psi')

In [38]:
phiD = lambda x : phi(x) * degrees
psiD = lambda x : psi(x) * degrees

In [29]:
state_centers = {
    'A' : [-150, 150], 
    'B' : [-70, 135], 
    'C' : [-150, -65], 
    'D' : [-70, -50], 
    'E' : [50, -100], 
    'F' : [40, 65]
}
interface_levels = {
    'A' : [10, 20, 45, 65, 80],
    'B' : [10, 20, 45, 65, 75],
    'C' : [10, 20, 45, 60],
    'D' : [10, 20, 45, 60],
    'E' : [10, 20, 45, 65, 80],
    'F' : [10, 20, 45, 65, 80],
    'alpha' : [10, 20, 45, 55, 65, 75],
    'beta' : [10, 20, 40, 45, 50, 60],    
}

In [30]:
def cvlines(data, *cvs):
    all_points = [cv(data) for cv in cvs]
    ret = []
    first = 0
    for d in range(1,len(data)):
        flip = False
        for c in range(len(cvs)):
            if periodicflip(all_points[c][d], all_points[c][d-1], math.pi):
                flip = True
                                
        if flip:
            ret.append([all_points[c][first:d] for c in range(len(cvs))])
            first = d

    ret.append([all_points[c][first:d+1] for c in range(len(cvs))])    
            
    return ret
            
    
def periodicflip(val1, val2, period):
    return (period**2 - (val1 - val2)**2) < (val1 - val2)**2

In [31]:
%config InlineBackend.figure_format='retina'

def plot_main():
    plt.figure(figsize=(8, 8))
    rect = plt.Rectangle((-180,-180),360,360, facecolor="none", edgecolor="none")    
    
    plt.xlim(-180, 180);
    plt.ylim(-180, 180);    
    
    centers = state_centers.values()

    plt.plot([x[0] for x in centers], [x[1] for x in centers], 'ko')

    fig = plt.gcf()

    for level in range(100,0,-1):
        for col in [True, False]:
            for name, center in state_centers.iteritems():
                if level == 1:
                    plt.annotate(name, xy=center, xytext=(center[0]+10 + 1, center[1] - 1), fontsize=20, color='k')
                    plt.annotate(name, xy=center, xytext=(center[0]+10, center[1]), fontsize=20, color='w')

                if level in interface_levels[name]:
                    for xp in [-1, 0, 1]:
                        for yp in [-1, 0, 1]:
                            if col:
                                circle = plt.Circle((center[0] + xp * 360, center[1] + yp * 360), level, color='w')
                                fig.gca().add_artist(circle)
                            else:
                                l = 1.0 * level / 100
                                circle = plt.Circle(
                                    (center[0] + xp * 360, center[1] + yp * 360), 
                                    level - 1, 
                                    color=(l + 0.0,l + 0.2, 0.6)
                                )
                                fig.gca().add_artist(circle)

                        # circle.set_clip_path(rect)

    plt.axis('equal')
    plt.axis((-180,180,-180,180))
                    
    plt.xlim(-180, 180);
    plt.ylim(-180, 180);    
    
    return fig

In [32]:
def plot_traj(traj):    
    angles = cvlines(traj, phi, psi)

    for angle in angles:
        plt.plot(degrees * np.array(angle[0])[:], degrees * np.array(angle[1])[:], 'ko', linewidth=0.5)
        plt.plot(degrees * np.array(angle[0])[:], degrees * np.array(angle[1])[:], 'k-', linewidth=0.5)

In [34]:
degrees = 180/3.14159 # psi reports in radians; I think in degrees

In [39]:
#! skip
mstis_calc.live_visualization = paths.LiveVisualization(mstis, phiD, psiD, [-180.0, 180.0], [-180.0, 180.0])
#mstis_calc.live_visualization.background = None
mstis_calc.live_visualization.background = plot_main()
mstis_calc.live_visualization.draw_background()
mstis_calc.visualize_frequency = 1 # increasing this number speeds things up, but isn't as pretty

Figure(640x640)


In [43]:
mstis_calc.live_visualization = None

Now everything is ready: let's run the simulation!

In [52]:
mstis_calc.run_until(10000)

Working on Monte Carlo cycle number 2260.



ValueError: Some coordinates became 'nan'; simulation is unstable or buggy.

In [32]:
# commented out during development, so we can "run all" and then do more
#storage.close()